# Convert `GeoTIFFs` in Google Cloud Storage to  Earth Engine `ImageCollections`

In [1]:
import numpy as np
import rasterio
import os
from google.cloud import storage

## Argentina SOC concentration dataset

**Input data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `SOC_YEAR_qQUANTILE_dDEPTH.tif`:

- YEAR: 1982-2017
- QUANTILE: 0.05,0.5,0.95 percentiles
- DEPTH:
    - 2.5 --> for the interval 0-5cm
    - 10 --> for the interval 5-15cm
    - 22.5 --> for the interval 15-30cm
    - 45 --> for the interval 30-60cm
    - 80 --> for the interval 60-100cm
    - 150 --> for the interval 100-200cm

All data are in geotiff in projection EPSG:4326.
The data are SOC * 10 in order to use integers and therefore save space.
To obtain the real SOC values the data need to be divided by 10 (or the units need to be changed).

**Output data location:**

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q5

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q50

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_concentration_q95

**Move objects**

In [2]:
def move_blob(bucket_name, privatekey_path, source_file_prefix, destination_path):
    """Move files in a bucket."""
    bucket_name = 'vizz-data-transfer'
    privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"
    
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)
    
    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name, prefix=source_file_prefix)
    
    for blob in blobs:
        new_name = blob.name.split('/')[-1]
        new_name =  destination_path + new_name
        print(blob.name)
        bucket.copy_blob(blob, destination_bucket = bucket, new_name = new_name)
        #bucket.delete_blob(blob.name)  

In [ ]:
bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

move_blob(bucket_name, privatekey_path, source_file_prefix='SOC_maps/SOC_', destination_path='SOC_maps/SOC_concentration/')

### Rearrange `.tif` files

We save the different depths as bands of the same image

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name, privatekey_path):
    """Uploads a file to the bucket."""
    storage_client = storage.Client.from_service_account_json(privatekey_path)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

In [ ]:
years = (np.arange(36)+1982).astype(np.str)
depths = ['2.5', '10', '22.5', '45', '80', '150']
quantiles = ['0.5']#['0.05', '0.5', '0.95']
base_url = 'https://storage.googleapis.com/vizz-data-transfer/SOC_maps/SOC_concentration/'

bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

for quantile in quantiles:
    print('Quantile', quantile)
    # Read metadata of first file
    with rasterio.open(base_url+'SOC_'+str(years[0])+'_q'+quantile+'_d'+depths[0]+'.tif') as src0:
        profile = src0.profile
    
    # Update metadata to reflect the number of bands
    profile.update(count = len(depths))

    for i, year in enumerate(years):
        
        with rasterio.open('../data/SOC_'+year+'_q'+quantile+'.tif', 'w', **profile) as dst:
            for id, depth in enumerate(depths, start=1):
                with rasterio.open(base_url+'SOC_'+year+'_q'+quantile+'_d'+depth+'.tif') as src1:
                    dst.write_band(id, src1.read(1))
                
        upload_blob(bucket_name, '../data/SOC_'+year+'_q'+quantile+'.tif', 'SOC_maps/SOC_concentration/SOC_'+year+'_q'+quantile+'.tif', privatekey_path)

        os.remove('../data/SOC_'+year+'_q'+quantile+'.tif')

### Create a GEE ImageCollection 

In [ ]:
years = (np.arange(36)+1982).astype(np.str)
quantiles = ['0.5'] #['0.05', '0.5', '0.95']
quant_per = ['50'] #['5', '50', '95']

for i, quantile in enumerate(quantiles):
    print('Quantile', quantile)
    quant_name = quant_per[i]
    # We create a collection asset in Earth Engine to use as a write target.
    !earthengine create collection projects/soils-revealed/experimental-dataset/SOC_concentration_q{quant_name}

    # Next, we use the upload image command to send individual images to the collection.
    for year in years:
        year = str(year)
    
        !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/experimental-dataset/SOC_concentration_q{quant_name}/SOC_concentration_{year} gs://vizz-data-transfer/SOC_maps/SOC_concentration/SOC_{year}_q{quantile}.tif           

## Argentina SOC stocks dataset

**Data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `Feb19_cstocks_YEAR_030_ll.tif``:
- YEAR: 1982-2017
- The stocks were calculated in the 0 to 30 cm interval. 

**Output data location:**

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_stock_0_30

**Move objects**

In [ ]:
bucket_name = 'vizz-data-transfer'
privatekey_path =  "/Users/ikersanchez/Vizzuality/Keys/soils-revealed/soc-platform-880dbb41449c.json"

move_blob(bucket_name, privatekey_path, source_file_prefix='SOC_maps/Feb19', destination_path='SOC_maps/SOC_stock/')

In [ ]:
years = (np.arange(36)+1982).astype(np.str)

# We create a collection asset in Earth Engine to use as a write target.
!earthengine create collection projects/soils-revealed/experimental-dataset/SOC_stock_0_30

# Next, we use the upload image command to send individual images to the collection.
for year in years:
    !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/experimental-dataset/SOC_stock_0_30/SOC_stock_{year} gs://vizz-data-transfer/SOC_maps/SOC_stock/Feb19_cstocks_{year}_030_ll.tif
        

## Argentina SOC stocks dataset

**Data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/

**Data description:**

The name structure of the files is `cstock030_YEAR_030cm_Q0.5.tif`:
- YEAR: 1982-2017
- The stocks were calculated in the 0 to 30 cm interval. 

**Output data location:**

https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOC_stock_EJSS

In [3]:
years = (np.arange(36)+1982).astype(np.str)

# We create a collection asset in Earth Engine to use as a write target.
!earthengine create collection projects/soils-revealed/experimental-dataset/SOC_stock

# Next, we use the upload image command to send individual images to the collection.
for year in years:
    !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/experimental-dataset/SOC_stock/SOC_stock_{year} gs://vizz-data-transfer/SOC_maps/SOC_stock_EJSS/cstock030_{year}_030cm_Q0.5.tif

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

W0914 18:22:42.424133 140736175551360 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ikersanchez/anaconda3/envs/geoenv/lib/python3.8/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above excep

## Soil carbon debt of 12,000 years of human land use
**Data location:**
- Current stock https://github.com/whrc/Soil-Carbon-Debt/blob/master/SOCS/SOCS_0_200cm_year_2010AD_10km.tif
- Historic stock https://github.com/whrc/Soil-Carbon-Debt/blob/master/SOCS/SOCS_0_200cm_year_NoLU_10km.tif

**Output data location:**

- Current stock https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOCS_0_200cm_year_2010AD_10km
- Historic stock https://code.earthengine.google.com/?asset=users/iker/SOC_maps/SOCS_0_200cm_year_NoLU_10km

## Future SOC stocks dataset

**Data location:**

https://storage.cloud.google.com/vizz-data-transfer/SOC_maps/Future/

**Data description:**

Scenario file are named scenario_xxxx_yyyy_Yzz

Where: 
  - xxxx = scenario names:
    - crop.MG 
    - crop.MGI
    - crop.I
    - grass.part
    - grass.full
    - rewilding
    - degradation.ForestToGrass
    - degradation.ForestToCrop
    - degradation.NoDeforestation
  - yyyy = either dSOC (for change in SOC stocks) or SOC (for absolute SOC stocks)
  - zz = years after change in land use or management (05, 10, 15, 20)

All scenario rasters are in units of Mg C / ha to 30 cm depth

**Scenarios**
- crop_MG: Cropland, improved management only 
- crop_MGI: Cropland, improved management and inputs 
- crop_I: Cropland, improved inputs only 
- grass-part: Grassland, partial restoration 
- grass-full: Grassland, full restoration 
- rewilding: Rewilding 
- degradation-ForestToGrass: Degradation (includes deforestation to degraded grassland condition)
- degradation-ForestToCrop: Degradation (includes deforestation to cropland)
- degradation-NoDeforestation: Degradation (no deforestation)

**Output data location:**

https://code.earthengine.google.com/?asset=projects/soils-revealed/Future/

### Upload to a GEE ImageCollection 

In [ ]:
#scenarios = ['crop_MG', 'crop_MGI', 'crop_I', 'grass_part', 'grass_full', 'rewilding', 'degradation_ForestToGrass', 'degradation_ForestToCrop', 'degradation_NoDeforestation']
scenarios = ['degradation_NoDeforestation']
years = {'2023': '05', '2028': '10', '2033': '15', '2038': '20'}
years = {'2038': '20'}

for scenario in scenarios:
    print('Scenario', scenario)
    # We create a collection asset in Earth Engine to use as a write target.
    #!earthengine create collection projects/soils-revealed/Future/scenario_{scenario}_dSOC
    
    for year in years:
        file_name = f'scenario_{scenario}_dSOC_Y{years[year]}'
        print('File name', file_name)
        !earthengine upload image --time_start {year}-01-01 --time_end {year}-12-31 --asset_id projects/soils-revealed/Future/scenario_{scenario}_dSOC/{file_name} gs://vizz-data-transfer/SOC_maps/Future/{file_name}.tif 